In [ ]:
# This code is Performs EPSO on a Discrete Graph
import pickle
import numpy as np
import random

# Loading the database
X = pickle.load(open('hydrogen_input_output.pkl', 'rb'))['x']
print("shape of X:", np.shape(X))

y = pickle.load(open('hydrogen_input_output.pkl', 'rb'))['y']
print("shape of y:", np.shape(y))

# A function to return the index of the input in the database
def input_index(Xinput):
  for i, input in enumerate(X):
    if np.array_equal(Xinput, input):
      return y[i]
  return None

shape of X: (98694, 7)
shape of y: (98694,)


In [ ]:
# The objective function essentially returns the output of the point in the database closest to it
def objective_function(x):
    x = np.array(x)

    # Finding the closest data point in the dataset
    distances = np.sum((X - x)**2, axis=1)
    closest_index = np.argmin(distances)

    return y[closest_index]

# snap_position essentially snaps the input to the point in the database closest to it
def snap_position(x):
    x = np.array(x)

    distances = np.sum((X - x)**2, axis=1)
    closest_index = np.argmin(distances)

    snapped_input = X[closest_index]
    return snapped_input


# We approach Evolutionary Particle Swarm Optimization from a Object-Oriented Perspective, since that's how the algorithm works: a bunch of particles (objects) talking to each other
class Particle:
    def __init__(self, num_dimensions, init_position, index):
        self.position = init_position
        self.velocity = np.zeros(num_dimensions)  # Initially the velocity of each particle is zero
        self.best_position = np.copy(self.position) # The best position seen by the particle initially is its initial position!
        self.best_fitness = objective_function(self.position)
        self.index = index  # the index variable allows us to label and keep track of each particle

# The EPSO class actually handles the running of the PSO algorithm
class EPSO:
    def __init__(self, num_particles, num_dimensions, lower_bound, upper_bound, w, c1, c2, num_iterations, init_positions):
        self.num_particles = num_particles  # No. of particles
        self.num_dimensions = num_dimensions  # The dimensions of the dataset
        self.lower_bound = lower_bound  # The lower bound for each dimension of the dataset (we normalize each dimension to lie within [0.0, 1.0])
        self.upper_bound = upper_bound  # Same as above, but upper bound instead
        self.w = w  # w is the weight parameter of each particle; it's a measure of its inertia
        self.c1 = c1  # c1 is a parameter that controls how much the global best position affects its velocity
        self.c2 = c2  # c2 is a parameter that controls how much its personal best position affects its velocity
        self.num_iterations = num_iterations  # Total number of iterations to run PSO for
        self.particles = [Particle(num_dimensions, init_positions[ind], ind) for ind in range(num_particles)] # Initializes a list of particles
        self.global_best_position = np.copy(self.particles[0].position) # global_best_position keeps track of the global best position of all particles. For now, we initialize it arbitrarily
        self.global_best_fitness = self.particles[0].best_fitness # The output of the global_best_position via the database. Again, we initialize it arbitrarily for now

    # This method handles updating each particle's parameters in each iteration
    def update_particles(self):
        for particle in self.particles:
            # Updating the velocity
            particle.velocity = (self.w * particle.velocity
                                 + self.c1 * np.random.rand() * (particle.best_position - particle.position)
                                 + self.c2 * np.random.rand() * (self.global_best_position - particle.position))

            # Updating the position
            particle.position = particle.position + particle.velocity

            # Cliping the position of each particle
            particle.position = np.clip(particle.position, self.lower_bound, self.upper_bound)  # Firstly clipping it so that it stays within the upper and lower bounds
            particle.position = snap_position(particle.position)  # And then snapping it to the closest position within the database


    # This method updates each particle's personal best and the global best
    def update_personal_and_global_bests(self):
        for particle in self.particles:
            # Evaluating the current output of the particle based on its position
            current_output = objective_function(particle.position)

            # Updating the particle's personal best:
            if (current_output > particle.best_fitness):
                particle.best_position = np.copy(particle.position)
                particle.best_fitness = current_output

                # Updating the global best position:
                if (current_output > self.global_best_fitness):
                    self.global_best_position = np.copy(particle.position)
                    self.global_best_fitness = current_output

            # This is mainly for debugging purposes; Feel free to comment out if it clutters the output too much.
            print("Particle #", particle.index, ": ")
            print("-> Current Position: ", particle.position)
            print("-> Current Velocity: ", particle.velocity)
            print("-> Current Output: ", input_index(particle.position))
            print("-> Personal Best Position: ",  particle.best_position)


        print("---> Global Best Position: ", self.global_best_position, " <---")
        print("---> Global Best Output: ", input_index(self.global_best_position), " <---")

    # This method eliminates the weakest performing particle(s) in each iteration
    def eliminate(self):
        # First, we want to decide how many particles we want to eliminate in each iteration.
        num_to_eliminate = int((self.num_particles / (self.num_iterations - 1)))

        # A quick sanity check; if there are no particles left to remove, leave the method!
        if not self.particles:
            return

        # Increment over all particles eliminate
        for _ in range(num_to_eliminate):
            # Find the particle with the least output, and determine its index
            min_index = 0
            for j in range(1, len(self.particles)):
                if (input_index(snap_position(self.particles[j].position)) < input_index(snap_position(self.particles[min_index].position))):
                    min_index = j

            # Remove that particle!
            self.particles.pop(min_index)
            print("eliminated index ", min_index)

     # This method handles the actual EPSO process:
    def optimize(self):
        for i in range(self.num_iterations):
            print("Iteration #", i, ": ")
            print("x-x-x-x-x-x-x-x-x-x-x")
            self.update_particles()
            self.update_personal_and_global_bests()

            # If we're not at the last iteration, eliminate the weakest performing particle(s)!
            if (i != (self.num_iterations - 1)):
                self.eliminate()
            print("Particles remaining after Iteration #", i, ": ", len(self.particles))  # For debugging; feel free to comment out

        return self.global_best_position, self.global_best_fitness


In [ ]:
# Adjusting Parameters
num_particles = 10
num_dimensions = 7 # Can be automated, but just easier to understand if manually entered
lower_bound = 0
upper_bound = 1.0
# EPSO parameters
w = 0.8
c1 = 0.8
c2 = 0.8
num_iterations = 10

# Randomizing initial positions of the particles
initial_positions_EPSO = random.choices(X, k=num_particles)
print("Initial Particle Positions: ", initial_positions_EPSO)

# Defining the EPSO object:
epso = EPSO(num_particles, num_dimensions, lower_bound, upper_bound, w, c1, c2, num_iterations, initial_positions_EPSO)
print("Beginning PSO Algorithm: ")
best_position, best_fitness = epso.optimize() # Running the optimize method

# Outputting the final results
print("Best position found: ", best_position)
print("Best fitness found: ", best_fitness)

Initial Particle Positions:  [array([0.1961165 , 0.13481846, 0.3420058 , 0.63636364, 0.01707249,
       0.15980902, 0.06713287]), array([0.11456311, 0.44628513, 0.6775936 , 0.75757576, 0.0338651 ,
       0.07470861, 0.11258741]), array([0.09902913, 0.36665333, 0.48645814, 0.7979798 , 0.04058214,
       0.18677152, 0.12657343]), array([0.11067961, 0.38110872, 0.55584953, 0.74747475, 0.03442485,
       0.12652717, 0.09188811]), array([0.12815534, 0.25213128, 0.42556983, 0.65656566, 0.02602855,
       0.09900295, 0.09132867]), array([0.33398058, 0.01055385, 0.04505609, 0.26262626, 0.00671704,
       0.0647381 , 0.05566434]), array([0.14174757, 0.27937026, 0.51612758, 0.70707071, 0.02574867,
       0.09675607, 0.08951049]), array([0.13203883, 0.35139179, 0.60816967, 0.77777778, 0.0302267 ,
       0.11290549, 0.08937063]), array([0.09320388, 0.45777128, 0.5661048 , 0.78787879, 0.04282116,
       0.14169358, 0.11188811]), array([0.25242718, 0.01976513, 0.06413234, 0.34343434, 0.00699692,
   